In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [ ]:
matches = pd.read_csv('nba_games_normalized.csv', index_col=0)
matches

In [ ]:
# cleaning up the data for use in the model
matches["date"] = pd.to_datetime(matches["Date"])
matches["venue_code"] = matches["Venue"].astype("category").cat.codes
matches["opp_code"] = matches["Opponent"].astype("category").cat.codes
matches["hour"] = matches["StartET"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["Result"] == "W").astype("int")

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
training_set = matches[matches["date"] < '2024-09-30']
test_set = matches[matches["date"] >= '2024-09-30']

In [ ]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [ ]:
# Predict && print metrics of initial model
rf.fit(training_set[predictors], training_set["target"])
predictions = rf.predict(test_set[predictors])
print("Accuracy:", accuracy_score(test_set["target"], predictions)) # whether it was right or wrong
print("Precision:", precision_score(test_set["target"], predictions)) # when it predicted win, how often was it correct

actual_and_pred = pd.DataFrame(dict(actual=test_set["target"], predicted=predictions))
pd.crosstab(index=actual_and_pred["actual"], columns=actual_and_pred["predicted"])


In [ ]:
# showing how to group by team (prints all LAL games)
grouped_matches = matches.groupby("Team")
group = grouped_matches.get_group('LAL')
group

In [ ]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, closed='left').mean() # rolling average of last 3 games, excluding current game
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols) # drop rows where rolling averages are NaN (first 2 games of each team)
  return group

In [ ]:
# create rolling average columns
cols = ["FGA", "FGM", "3PA", "3PM", "3P%", "FTA", "FTM", "FT%"]
new_cols = [f"rolling_{c}" for c in cols]

In [ ]:
# test rolling_averages by showing rolling averages for LAL
rolling_averages(group, cols, new_cols)

In [ ]:
parts = []

for team, group in matches.groupby("Team", sort=False):
    # workaround for groupby().apply() FutureWarning
    part = rolling_averages(group.copy(), cols, new_cols)
    parts.append(part)

matches_rolling = pd.concat(parts, ignore_index=True)
matches_rolling

In [ ]:
def make_predictions(data, predictors):
  training_set = data[data["date"] < '2024-09-30']
  test_set = data[data["date"] >= '2024-09-30']
  rf.fit(training_set[predictors], training_set["target"])
  predictions = rf.predict(test_set[predictors])
  combined = pd.DataFrame(dict(actual=test_set["target"], predicted=predictions), index=test_set.index)
  precision = precision_score(test_set["target"], predictions)
  return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)
precision

In [ ]:
combined = combined.merge(matches_rolling[["Date", "Team", "Opponent", "Result"]], left_index=True, right_index=True)
combined

In [ ]:
# Combine home and away predictions
merged = combined.merge(combined, left_on=["Date", "Team"], right_on=["Date", "Opponent"])
merged.head()

In [ ]:
# when model predicts team A would win && team B would lose, what happened?
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()